## Importing packages
If you are working with a notebook, it is a good practice to import all the packages that you need at the top of the notebook. This will automatically give you a rough idea of what the notebook is about, and you will avoid random package imports throughout the notebook.

- Import the three packages you have used so far with their correct renaming conventions.

## Reading csv files
The package pandas introduces the function `pd.read_csv()` that allows you to turn a csv file directly into a data frame. Depending on the specific format of the csv file, you have to use quite a few arguments to `pd.read_csv()` to read the csv file without any errors. See the following arguments that are useful/required to load the temperature csv files:
- `sep`: separator of the data columns, this will be a comma in most cases but sometimes this is also a semicolon
- `decimal`: decimal character for numerical data, this will be a comma if the file uses a "German" number format
- `names`: list of names to use for the columns, use this if the column names are not (well) specified in the csv file
- `usecols`: list of columns to use, if there are six columns in the csv file but you don't need the first and the last one, use `[1, 2, 3, 4]`
- `index_col`: specify the column to be used as the index, this is applied after `usecols`
- `skiprows`: skip the specified number/list of rows before starting to read the data columns, use this if there is some metadata at the top of your csv file
- `header`: index of the row where the column names are specified, this is counted after applying `skiprows`
- `nrows`: specify the number of rows to load, use this if you only want to load the entire csv file at once
- `encoding`: to save text in a file you need an encoding that specifies how strings are transformed to bytes, the default here should be `"utf-8"`

In any case, you should always open a csv file in an editor first to check the format. Unless you already know the arguments for the specific format, this will always be quicker than trying to blindly read the file with pandas.

- Look at the csv file for the power consumption data in editor. What is used as the `delimiter` and the `decimal`? Do you have to skip any rows?
- Use the argument `index_col` to directly use the time data as the index.
- Specify the number of columns before loading the data frame or remove the empty column after loading the file.
- Manually set the column names or rename the columns to get rid of the unit `[kW]` in the column names.

## Converting the index to datetime objects
The index of the data frame already looks like a datetime but these are actually just strings. If you want to properly use the index column, you have to convert it to datetime objects. Pandas provides the function `pd.to_datetime()` that will try to interpret strings/numbers as datetime objects. It might be necessary to further specify the format since this is often stored in the local time format. E.g. in Germany we use the format `day.month.year` whereas the everyday format in the US is `month/day/year`. As long as the format does not change throughout the data frame, you can always find a way to convert this to datetime objects. It might however be necessary to manually specify the datetime format. Look at the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) to learn more about the options to interpret strings/numbers as datetime objects.

- Convert the index of the data frame to a DatetimeIndex with the function `pd.to_datetime()`. You can directly reassign the DatetimeIndex to `df.index`.
- What is the type of the individual values in the DatetimeIndex? Find out how you can get the hour, minute, day of week from an individual value  and from the entire DatetimeIndex.
- EXTRA: Add the `hour` and the `dayofweek` of every value in the DatetimeIndex as new columns to the data frame. This will allow easier grouping/querying of the data.

## Power consumption over one week
You can index the time with a string (range) to select only a fraction of the data frame. If you only want to index the time by the days, you can omit the hours. The hours will then default to midnight of the respective days. See the following example to select all rows from the days `"2023-10-01"` and `"2023-10-02"`:
```python
df.loc["2023-10-01":"2023-10-02"]
```
For such a time range, the stop will now be included. If you look at the rows in the data frame selection above, the column time will go from `2023-10-01 00:00:00` to `2023-10-02 23:00:00`.

- Reproduce the example above to get the data for two days. Change the index range such that the data only starts at noon on the first day.
- Select the data from the last week (starting on `"2023-10-09"`) and plot the column `"P_tot"`. There should be five peaks in your data and two constant days. Label the axes (change the xlabel if you don't like it) and add a title.
- Look at the data from one week earlier (starting on `"2023-10-02"`) and plot the column `"P_tot"` again. Why does the second day of the week have a much smaller power consumption?

## Highest power consumption during the day/night
For this exercise you can assume that the day lasts from 07:00 to 22:00 (including 22:00) and the night lasts from 23:00 to 06:00 (including 06:00). Define two query strings that represent these conditions. As an example, if your column is called `"hour"`, you would use the following query to select everything up to 12:00 (including 12:00)
```python
query_morning = "hour <= 12"
df.query(query_morning)
```
You can combine conditions with `and`/`or`. The usage of parentheses in the queries is optional if you only have two conditions. You can however always use them to improve the readability of the queries.  

- Apply the two queries for the day and the night to the data frame. What is the maximum value of the column `"P_tot"` during the day/the night?
- Find the index/timestamp where the highest power consumptions were measured with the method `.idxmax()`. Save these data points in new variables for later use. They should be on the days `"2023-05-11"` and `"2023-05-12"`.
- Display the total power consumption in the range from `"2023-05-10"` to `"2023-05-13"`. Add the maximum data points to the plot with `plt.scatter()` and label them. Use different marker shapes or different colors for the day/night maximum.
- If you look at the maximum data point of the night in the plot, would you use different time ranges to separate the day and the night?

## Power consumption as a function of the hour
If you look at the power consumption over a few days, you can clearly see that the consumption varies a lot throughout a regular weekday. To take a closer look at the average changes over a day, you want to group all the data by the column `"hour"`. You can then compute the average of the columns per unique value of the `"hour"`. The averaged data frame will then have the `"hour"` as the index.

- Use the column `"dayofweek"` to write a query that selects only the weekdays. If you are not sure on which day the week starts, look at the data from this week and compare the dates to the values in the column `"dayofweek"`.
- How many data points did you remove from the original data frame with the weekday query?
- Select only the weekdays with a query and use the data frame method `.groupby()` to group the data by the column `"hour"`, and store this group as a new variable. You can then use the method `.mean()` to directly compute the mean values of all data frame columns for each hour.